<a href="https://colab.research.google.com/github/Byeon-MJ/DL_Computer_Vision/blob/main/Drowsiness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drowsiness Detection
- 얼굴에서 Face Landmark 찾아내고 Eyes Landmark를 이용해 눈을 뜨고있는지 감고있는지 감지
- Eyes Landmark 점간의 거리로 EAR(Eye Aspect Ratio) 계수를 계산하여 판단
- EAR 계수가 0.2 ~ 0.3 보다 크면 눈을 뜨고있다고 판단

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import dlib
import cv2
import time

In [ ]:
# eye landmark 정의
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

In [ ]:
# 얼굴을 감지하는 haarcascade feature 파일
face_cascade_name = '/content/gdrive/MyDrive/CV/Face Detection/data/haarcascades/haarcascade_frontalface_alt.xml'

# 얼굴 감지 객체 생성
face_cascade = cv2.CascadeClassifier()

if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!) Error loading face cascade')
    exit(0)

In [ ]:
# 이미 학습된 dlib 모델
predictor_file = '/content/gdrive/MyDrive/CV/Face Landmark/shape_predictor_68_face_landmarks.dat'

# facial landmark 찾는 객체 생성
predictor = dlib.shape_predictor(predictor_file)

# 원본 동영상 파일
file_name = '/content/gdrive/MyDrive/CV/Drowsiness Detection/drowsiness.mp4'
# output 동영상 이름
output_name = 'drowsiness_detection_video.mp4'

# 변수 초기화
status = 'Awake'    
number_closed = 0   
min_EAR = 0.21      # 눈 뜬 간격 최소화 설정
closed_limit = 7    # 눈을 감은 patience 수
txt = None          # text 문구 초기화
color = None        # text 문구 글자색 초기화
elapsed_time = 0    # 동영상 detection 하는데 걸린 총 경과시간 초기화

In [ ]:
# EAR(eye aspect ratio) 계산 함수 정의
def getEAR(points):
    A = np.linalg.norm(points[1] - points[5])
    B = np.linalg.norm(points[2] - points[4])
    C = np.linalg.norm(points[0] - points[3])
    return (A + B) / (2.0 * C)

In [ ]:
def detectAndDisplay(image):
    global number_closed
    global color
    global show_frame
    global txt
    global elapsed_time

    # largest_box 초기화
    largest_box = 0

    start_time = time.time()

    frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    
    # face detection
    faces = face_cascade.detectMultiScale(frame_gray)

    # detect된 box 중에서 largest box 찾기
    for (_x, _y, _w, _h) in faces:
        SZ = _w * _h
        if SZ > largest_box:
            largest_box = SZ
            x, y, w, h = _x, _y, _w, _h

    # box 시작 위치 좌표 저장
    x_p, y_p = int(x), int(y)

    # 이미지에 box 그리기
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
    # dlib에서 사용하는 rect 좌표형식
    rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))

    # 68개 point 좌표
    points = np.matrix([[p.x, p.y] for p in predictor(frame_gray, rect).parts()])
    show_parts = points[EYES]   # EYES points
    right_eye_EAR = getEAR(points[RIGHT_EYE])           # 오른쪽 눈 EAR 계수
    left_eye_EAR = getEAR(points[LEFT_EYE])             # 왼쪽 눈 EAR 계수
    mean_eye_EAR = (right_eye_EAR + left_eye_EAR) / 2   # 양쪽 눈 EAR 계수 평균값

    # 오른쪽 눈 중앙 계산
    right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype('int')
    # 왼쪽 눈 중앙 계산
    left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype('int')
    
    # 오른쪽 눈 EAR 계수 Display
    cv2.putText(image, f'{right_eye_EAR:.2f}', (right_eye_center[0, 0], right_eye_center[0, 1] + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    # 왼쪽 눈 EAR 계수 Display
    cv2.putText(image, f'{left_eye_EAR:.2f}', (left_eye_center[0, 0], left_eye_center[0, 1] + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # 이미지에 눈을 표현해주기 위해 landmark point 그리기
    for (i, point) in enumerate(show_parts):
        x = point[0, 0]
        y = point[0, 1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

    if mean_eye_EAR > min_EAR:  # 눈을 뜨고 있는 경우
        color = (0, 255, 0)
        status = 'Awake'
        number_closed -= 1
        if (number_closed < 0):
            number_closed = 0
    else:                       # 눈을 감고 있는 경우
        color = (0, 0, 255)
        status = 'Sleep'
        number_closed += 1
        txt = status + ', Sleep count : ' + str(number_closed) + ' / ' + str(closed_limit)
        cv2.putText(image, txt, (x_p+3, y_p-7), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # video 불러오기
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(output_name, fourcc, 30, (image.shape[1], image.shape[0]), True)

    # disk에 image write
    if writer is not None:
        writer.write(image)

    # frame 당 처리 시간
    frame_time = time.time() - start_time
    print(f'Frame time {frame_time:.3f} seconds')
    # 총 경과 시간 누적
    elapsed_time += frame_time

In [ ]:
cap = cv2.VideoCapture(file_name)

writer = None

# 동영상 확인
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

In [ ]:
while True:
    # 원본 동영상에서 frame 읽기
    ret, image = cap.read()

    # 원본 동영상에서 더이상 frame을 읽지 못한다면 Exit
    if image is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print(f'elapsed time {elapsed_time:.3f} seconds')
        break
    
    detectAndDisplay(image)